# Imports

In [91]:
# Allow importing items in src folder
import sys

SCRIPT_DIR = "/home/shane/Projects/sign_data"
sys.path.append(SCRIPT_DIR)

In [92]:
from src.files import load_pickle, read_from_file
from src.models import Cnn_3d, load_model
from src.inference import load_video, predict_signs
from src.utilities import save_to_csv

# Variables

In [93]:
model_path = '../trained_models/72_stream_cnn_resnet18_3d_20220508.pt'
normalization_stats = '../trained_models/normalization_stats_mhwl.pkl'
vocab_file = '../vocab.csv'
video_fname = '../videos/Alphabet50.mp4'
label_folder = '../video_labels/'

In [94]:
model_name = Cnn_3d

# Code

In [95]:
def group_predictions(data):
    grouped_data = []
    start_time = data[0][0]
    cur_sign = data[0][1]
    sign_length = 0
    for idx, frame in enumerate(data):
        sign_length += 1
        if frame[1] != cur_sign:
            end_time = data[idx - 1][0]
            if cur_sign != 'NaS' and sign_length > 12:
                grouped_data.append([round(start_time/1000, 2), round(end_time/1000, 2), cur_sign])
            start_time = frame[0]
            cur_sign = frame[1]
            sign_length = 0
    return grouped_data

# Get Video Predictions

In [96]:
# Load the normalization stats
normalization_stats = load_pickle(normalization_stats)

# Load vocab
vocab = read_from_file(vocab_file)

# Load the model
model = load_model(model_name, len(vocab), model_path)

# Open the video
video = load_video(video_fname)

# Loop over the frames of the video & process
predictions = predict_signs(video, model, vocab, normalization_stats,
                            frames_per_sign=30, number_of_coords=126,
                            display_pred=False)

In [97]:
grouped_preds = group_predictions(predictions)

In [98]:
label_file_name = label_folder + video_fname.split("/")[-1].split(".")[0] + ".csv"
label_file_name

'../video_labels/Alphabet50.csv'

In [99]:
save_to_csv(grouped_preds, label_file_name, headers=['start_time', 'end_time', 'word'])

Make sure to manually check the output here - this isn't very accurate.

Run `ffplay ${video_path}` to do that